In [1]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import LinearRegression
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit
from pyspark.ml.feature import VectorAssembler
import matplotlib.pyplot as plt
from pyspark.sql.window import Window
import pyspark.sql.functions as F
from pyspark.sql.functions import first

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

# 5. Linear Regression Analysis

## 5.1 Regression with RF metrics

In [5]:
venmo_reg = spark.read.parquet('/FileStore/tables/venmo_reg.parquet')

In [6]:
# Change the dataframe into the model suitable format
vectorAssembler = VectorAssembler(inputCols = ['recency', 'frequency'], outputCol = 'features')
venmo_reg_agg = vectorAssembler.transform(venmo_reg)

In [7]:
# Train and test split
for i in range(0,13):
    splits = venmo_reg_agg[venmo_reg_agg['month']==i].randomSplit([0.7, 0.3])
    train = splits[0].select(['features','Y'])
    test = splits[1].select(['features','Y'])
    train_df = train.withColumnRenamed('Y','label')
    test_df = test.withColumnRenamed('Y','label')
    train_df.write.mode('overwrite').format("parquet").save('dbfs:/FileStore/tables/train_rf_month'+str(i)+'.parquet')
    test_df.write.mode('overwrite').format("parquet").save('dbfs:/FileStore/tables/test_rf_month'+str(i)+'.parquet')

In [8]:
MSE_list = []
for i in range(0,13):  
    # For each user’s lifetime point, regress recency and frequency on Y. 
    lr = LinearRegression(featuresCol='features', labelCol='label', maxIter=10)

    # We use a ParamGridBuilder to construct a grid of parameters to search over.
    # TrainValidationSplit will try all combinations of values and determine best model using the evaluator.
    paramGrid = ParamGridBuilder()\
        .addGrid(lr.regParam, [0.1, 0.01]) \
        .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])\
        .build()

    # A TrainValidationSplit requires an Estimator, a set of Estimator ParamMaps, and an Evaluator.
    tvs = TrainValidationSplit(estimator=lr,
                               estimatorParamMaps=paramGrid,
                               evaluator=RegressionEvaluator(),
                               # 80% of the data will be used for training, 20% for validation.
                               trainRatio=0.8)

    # Run TrainValidationSplit, and choose the best set of parameters.
    train_df = spark.read.parquet('/FileStore/tables/train_rf_month'+str(i)+'.parquet')
    test_df = spark.read.parquet('/FileStore/tables/test_rf_month'+str(i)+'.parquet')
    model = tvs.fit(train_df)
    
    # Out of Sample Mean Squared Error
    lr_predictions = model.transform(test_df)
    evaluator = RegressionEvaluator(metricName="mse")
    MSE = evaluator.evaluate(lr_predictions)
    MSE_list.append(MSE)
    
    # Print the performance of the models
    print('month'+str(i))
    print(model.bestModel._java_obj.getRegParam())
    print(model.bestModel._java_obj.getElasticNetParam())
    print("Coefficients: " + str(model.bestModel.coefficients))
    r2_evaluator = RegressionEvaluator(predictionCol="prediction",labelCol="label",metricName="r2")
    print("R Squared (R2) on test data = %g" % r2_evaluator.evaluate(lr_predictions))

month0
0.1
0.0
Coefficients: [0.0,0.0]
R Squared (R2) on test data = -1.66957e-06
month1
0.01
0.0
Coefficients: [-0.012163264797337653,51.705914326380004]
R Squared (R2) on test data = 0.158566
month2
0.1
0.0
Coefficients: [-0.012516396332146962,84.73639705152652]
R Squared (R2) on test data = 0.287831
month3
0.01
1.0
Coefficients: [-0.005539377956777105,131.5288464970313]
R Squared (R2) on test data = 0.408083
month4
0.01
0.5
Coefficients: [-0.004645671930652861,164.55394152613945]
R Squared (R2) on test data = 0.51264
month5
0.01
0.0
Coefficients: [-0.0036378338582666423,196.11840484561958]
R Squared (R2) on test data = 0.610806
month6
0.01
0.5
Coefficients: [-0.0028021081448836244,223.8832477386116]
R Squared (R2) on test data = 0.703187
month7
0.01
1.0
Coefficients: [-0.002107536497040002,249.56139351798524]
R Squared (R2) on test data = 0.776677
month8
0.01
1.0
Coefficients: [-0.0017134402059357772,271.4612664097339]
R Squared (R2) on test data = 0.837686
month9
0.01
0.0
Coefficients: [-0.00143475943185381,292.98768066763836]
R Squared (R2) on test data = 0.892668
month10
0.01
0.0
Coefficients: [-0.0009593748847321394,315.88733806559486]
R Squared (R2) on test data = 0.937062
month11
0.01
0.5
Coefficients: [-0.00046081246104289446,337.48087020508694]
R Squared (R2) on test data = 0.973601
month12
0.01
1.0
Coefficients: [0.0,358.2673716821002]
R Squared (R2) on test data = 0.999977

In [9]:
month_list = [0,1,2,3,4,5,6,7,8,9,10,11,12]
MSE_list_RF = MSE_list
fig, ax = plt.subplots()
ax.plot(month_list, MSE_list_RF)

ax.set(xlabel='month of lifetime', ylabel='MSE',
       title=' MSE for each lifetime point - RF')
ax.grid()

plt.show()

## 5.2 Regression with RF metrics and spending behavior profile metrics

In [11]:
accum_profile_prop = spark.read.parquet('/FileStore/tables/accum_profile_prop.parquet')
accum_profile_prop.show()

+----+-----+-------------+---------+------------+-----------+-----------+------------+---------+----------+-----------+-----------+
user|month|activity_perc|food_perc|illegal_perc|others_perc|people_perc|utility_perc|cash_perc|event_perc|transp_perc|travel_perc|
+----+-----+-------------+---------+------------+-----------+-----------+------------+---------+----------+-----------+-----------+
 2| 0| 0.0| 0.0| 0.0| 1.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0|
 2| 1| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0|
 2| 2| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0|
 2| 3| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0|
 2| 4| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0|
 2| 5| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0|
 2| 6| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0|
 2| 7| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0|
 2| 8| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0|
 2| 9| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0|
 2| 10| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0|
 2| 11| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0|
 2| 12| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0|
 3| 0| 0.0| 0.0| 0.0| 0.5| 0.5| 0.0| 0.0| 0.0| 0.0| 0.0|
 3| 1| 0.0| 0.17| 0.0| 0.67| 0.0| 0.17| 0.0| 0.0| 0.0| 0.0|
 3| 2| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0|
 3| 3| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0|
 3| 4| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0|
 3| 5| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0|
 3| 6| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0| 0.0|
+----+-----+-------------+---------+------------+-----------+-----------+------------+---------+----------+-----------+-----------+
only showing top 20 rows

In [12]:
venmo_reg_sb = venmo_reg.join(accum_profile_prop,['user','month'],how ='inner').orderBy('user','month').cache()

In [13]:
# Change the dataframe into the model suitable format
vectorAssembler = VectorAssembler(inputCols = ['activity_perc','food_perc','illegal_perc','others_perc','people_perc','utility_perc','cash_perc','event_perc','transp_perc','travel_perc','recency','frequency'], outputCol = 'features')
venmo_reg_sb_agg = vectorAssembler.transform(venmo_reg_sb)

In [14]:
# Train and test split
for i in range(0,13):
    splits = venmo_reg_sb_agg[venmo_reg_sb_agg['month']==i].randomSplit([0.7, 0.3])
    train = splits[0].select(['features','Y'])
    test = splits[1].select(['features','Y'])
    train_df = train.withColumnRenamed('Y','label')
    test_df = test.withColumnRenamed('Y','label')
    train_df.write.mode('overwrite').format("parquet").save('dbfs:/FileStore/tables/train_rf_sb_month'+str(i)+'.parquet')
    test_df.write.mode('overwrite').format("parquet").save('dbfs:/FileStore/tables/test_rf_sb_month'+str(i)+'.parquet')

In [15]:
MSE_list = []
for i in range(0,13):  
    # For each user’s lifetime point, regress recency, frequency AND her spending behavior profile on Y.
    lr = LinearRegression(featuresCol='features', labelCol='label', maxIter=10)

    # We use a ParamGridBuilder to construct a grid of parameters to search over.
    # TrainValidationSplit will try all combinations of values and determine best model using the evaluator.
    paramGrid = ParamGridBuilder()\
        .addGrid(lr.regParam, [0.1, 0.01]) \
        .addGrid(lr.fitIntercept, [False, True])\
        .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])\
        .build()

    # A TrainValidationSplit requires an Estimator, a set of Estimator ParamMaps, and an Evaluator.
    tvs = TrainValidationSplit(estimator=lr,
                               estimatorParamMaps=paramGrid,
                               evaluator=RegressionEvaluator(),
                               # 80% of the data will be used for training, 20% for validation.
                               trainRatio=0.8)

    # Run TrainValidationSplit, and choose the best set of parameters.
    train_df = spark.read.parquet('/FileStore/tables/train_rf_sb_month'+str(i)+'.parquet')
    test_df = spark.read.parquet('/FileStore/tables/test_rf_sb_month'+str(i)+'.parquet')
    model = tvs.fit(train_df)
    
    # Out of Sample Mean Squared Error
    lr_predictions = model.transform(test_df)
    evaluator = RegressionEvaluator(metricName="mse")
    MSE = evaluator.evaluate(lr_predictions)
    MSE_list.append(MSE)
    
    # Print the performance of the models
    print('month'+str(i))
    print(model.bestModel._java_obj.getRegParam())
    print(model.bestModel._java_obj.getElasticNetParam())
    print("Coefficients: " + str(model.bestModel.coefficients))
    r2_evaluator = RegressionEvaluator(predictionCol="prediction",labelCol="label",metricName="r2")
    print("R Squared (R2) on test data = %g" % r2_evaluator.evaluate(lr_predictions))

month0
0.01
0.0
Coefficients: [-0.2718377311050524,0.14802072384399367,0.3445872998770043,0.038880153445446,0.049291588295667654,0.06539602880225859,-0.02100334954875013,-0.23172918164512982,0.0146270997446884,-0.352793500235136,0.0,0.0]
R Squared (R2) on test data = 0.00195931
month1
0.01
0.5
Coefficients: [-0.11081267470807897,0.1901808176544087,0.3774001229703792,0.07258280846759024,0.025059847071407914,0.0,0.0,0.0,-0.01975330433077384,-0.39963026661441187,-0.01007709947108743,50.820602716438366]
R Squared (R2) on test data = 0.16106
month2
0.01
0.0
Coefficients: [-0.2758073291470771,0.12286014972480848,0.2745859852809904,-0.010226248739030339,-0.06074962954798243,-0.169528254744854,-0.08301018266437143,-0.26202381810729775,-0.0949426627990662,-0.3107138350294597,-0.009138248301958361,92.99608229135849]
R Squared (R2) on test data = 0.291668
month3
0.01
0.0
Coefficients: [-0.09460457795635614,0.19587727841863678,0.27774238208949636,0.08535076045269513,0.011219623325889614,-0.04969962759646838,0.03232689124136527,-0.09627156909402781,0.0002862753443676379,-0.3603807028326082,-0.004874194141615792,133.0766233020884]
R Squared (R2) on test data = 0.414333
month4
0.01
0.0
Coefficients: [0.0020756826686277284,0.2496458483418363,0.3150470277878055,0.1462268823281719,0.08300036081953208,-0.04140093548825411,0.29871898950871634,-0.1135503040303042,0.11544955220371758,-0.3209422878597891,-0.003779615045855291,166.1465781664881]
R Squared (R2) on test data = 0.513895
month5
0.01
0.0
Coefficients: [0.07071422358578901,0.262652593469514,0.31014133424906876,0.16600667115839302,0.11797207679452935,0.07244497111876697,0.333308071662513,-0.022716519255846924,0.09414104408979267,-0.17849002744676962,-0.0028829529212184755,196.726942618267]
R Squared (R2) on test data = 0.61529
month6
0.01
0.5
Coefficients: [0.0,0.22014928103391762,0.25972678423730855,0.16660812234823852,0.09121694401656805,0.05917744338440514,0.0,0.0,0.04638720644461195,0.0,-0.002320695597012894,224.04090155271535]
R Squared (R2) on test data = 0.704407
month7
0.01
0.0
Coefficients: [0.2666089161475276,0.4058012746667645,0.45093602623794093,0.3456754959017462,0.3041227303507604,0.28400935465631494,0.4418740238320594,0.27021625870705324,0.317549596435075,0.1916142008092149,-0.00016113369221243843,261.30277719969285]
R Squared (R2) on test data = 0.778351
month8
0.01
0.0
Coefficients: [0.11563377449195893,0.27788248470514487,0.2938890102895258,0.21645617754927637,0.19667594076505068,0.1634386977912109,0.2032302102100413,0.09721507006646467,0.1520007991565336,0.033663982217180026,-0.0015270985742170987,269.5319063156692]
R Squared (R2) on test data = 0.839813
month9
0.01
0.0
Coefficients: [0.1595762510462957,0.2661552000022101,0.29375350888134305,0.21128150481329314,0.18228131671057846,0.17074075024812466,0.1527105557471645,0.12380357545048438,0.21702111483493863,0.11117852452746911,-0.0010496655459023225,293.9396100246997]
R Squared (R2) on test data = 0.895321
month10
0.01
0.0
Coefficients: [0.11840198466215791,0.21903470564299,0.21394043142333743,0.18062859159444672,0.15908348319967988,0.1477688445538401,0.2073901011848039,0.0761695898679784,0.17636732426706758,0.12056585954265557,-0.0007759345218045366,314.3588942806715]
R Squared (R2) on test data = 0.937709
month11
0.01
0.0
Coefficients: [0.07909388401780522,0.1300205985045573,0.1497487781351109,0.11478673150016493,0.10977266616890888,0.09705052641318652,0.07375760458633142,0.07365936278915507,0.13585842834426018,0.10537014204728207,-0.0004117687539025561,336.2818665842955]
R Squared (R2) on test data = 0.974033
month12
0.01
0.5
Coefficients: [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.848995289351108e-06,359.19674489556354]
R Squared (R2) on test data = 0.99999

In [16]:
month_list = [0,1,2,3,4,5,6,7,8,9,10,11,12]
MSE_list_RF_SB  = MSE_list
fig, ax = plt.subplots()
ax.plot(month_list, MSE_list_RF_SB,color='tab:orange',label='RF and Spending Profile')

ax.set(xlabel='month of lifetime', ylabel='MSE',
       title=' MSE for each lifetime point - RF and Spending profile')
ax.grid()
ax.legend()

plt.show()

## 5.3 Regression with social network metrics

In [18]:
# Load the dataset
friend_df = spark.read.parquet('/FileStore/tables/dynamic_fof_degree.parquet')
cluster_coef_df = spark.read.parquet('/FileStore/tables/cluster_coef_final.parquet')
page_rank_df = spark.read.parquet('dbfs:/FileStore/tables/pagerank_df.parquet')

In [19]:
friend_df.createOrReplaceTempView("friend_df")
venmo_reg.createOrReplaceTempView("venmo_reg")
venmo_reg_sn = spark.sql("""
                SELECT v.user, v.month, v.Y, v.recency, v.frequency, 
                ifnull(f.degree,0) as friend_degree, 
                ifnull(f.fof_degree,0) as fof_degree
                FROM venmo_reg v LEFT JOIN friend_df f
                ON v.user = f.user AND v.month = f.month_passed
                """)
venmo_reg_sn.coalesce(1).write.mode('overwrite').format("parquet").save('dbfs:/FileStore/tables/venmo_reg_sn.parquet')

In [20]:
venmo_reg_sn = spark.read.parquet('/FileStore/tables/venmo_reg_sn.parquet')
venmo_reg_sn.createOrReplaceTempView("venmo_reg_sn")
cluster_coef_df.createOrReplaceTempView("cluster_coef_df")
page_rank_df.createOrReplaceTempView("page_rank_df")
venmo_reg_sn_1 = spark.sql("""
                SELECT v.*,
                ifnull(c.cc,0) as clustering_coefficient,
                ifnull(p.pagerank,0) as pagerank
                FROM venmo_reg_sn v LEFT JOIN cluster_coef_df c
                ON v.user = c.user AND v.month = c.month_passed
                LEFT JOIN page_rank_df p
                ON v.user = p.user AND v.month = p.month_passed
                """)
venmo_reg_sn_1.sort('user','month').coalesce(1).write.mode('overwrite').format("parquet").save('dbfs:/FileStore/tables/venmo_reg_sn_1.parquet')

In [21]:
# Change the dataframe into the model suitable format
venmo_reg_sn_1 = spark.read.parquet('/FileStore/tables/venmo_reg_sn_1.parquet')
vectorAssembler = VectorAssembler(inputCols = ['friend_degree','fof_degree','clustering_coefficient','pagerank'], outputCol = 'features')
venmo_reg_sn_agg = vectorAssembler.transform(venmo_reg_sn_1)

In [22]:
# Train and test split
for i in range(0,13):
    splits = venmo_reg_sn_agg[venmo_reg_sn_agg['month']==i].randomSplit([0.7, 0.3])
    train = splits[0].select(['features','Y'])
    test = splits[1].select(['features','Y'])
    train_df = train.withColumnRenamed('Y','label')
    test_df = test.withColumnRenamed('Y','label')
    train_df.write.mode('overwrite').format("parquet").save('dbfs:/FileStore/tables/train_sn_month'+str(i)+'.parquet')
    test_df.write.mode('overwrite').format("parquet").save('dbfs:/FileStore/tables/test_sn_month'+str(i)+'.parquet')

In [23]:
MSE_list = []
for i in range(0,13):  
    # For each user’s lifetime point, regress her social network metrics on Y. 
    lr = LinearRegression(featuresCol='features', labelCol='label', maxIter=10)

    # We use a ParamGridBuilder to construct a grid of parameters to search over.
    # TrainValidationSplit will try all combinations of values and determine best model using the evaluator.
    paramGrid = ParamGridBuilder()\
        .addGrid(lr.regParam, [0.1, 0.01]) \
        .addGrid(lr.fitIntercept, [False, True])\
        .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])\
        .build()

    # A TrainValidationSplit requires an Estimator, a set of Estimator ParamMaps, and an Evaluator.
    tvs = TrainValidationSplit(estimator=lr,
                               estimatorParamMaps=paramGrid,
                               evaluator=RegressionEvaluator(),
                               # 80% of the data will be used for training, 20% for validation.
                               trainRatio=0.8)

    # Run TrainValidationSplit, and choose the best set of parameters.
    train_df = spark.read.parquet('/FileStore/tables/train_sn_month'+str(i)+'.parquet')
    test_df = spark.read.parquet('/FileStore/tables/test_sn_month'+str(i)+'.parquet')
    model = tvs.fit(train_df)
    
    # Out of Sample Mean Squared Error
    lr_predictions = model.transform(test_df)
    evaluator = RegressionEvaluator(metricName="mse")
    MSE = evaluator.evaluate(lr_predictions)
    MSE_list.append(MSE)
    
    # Print the performance of the models
    print('month'+str(i))
    print(model.bestModel._java_obj.getRegParam())
    print(model.bestModel._java_obj.getElasticNetParam())
    print("Coefficients: " + str(model.bestModel.coefficients))
    r2_evaluator = RegressionEvaluator(predictionCol="prediction",labelCol="label",metricName="r2")
    print("R Squared (R2) on test data = %g" % r2_evaluator.evaluate(lr_predictions))

/databricks/spark/python/pyspark/ml/util.py:791: UserWarning: Can not find mlflow. To enable mlflow logging, install MLflow library from PyPi.
 warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)
month0
0.01
0.5
Coefficients: [0.8030585124558764,-0.06105958525544698,0.0,1407812.1801671316]
R Squared (R2) on test data = 0.0869281
month1
0.01
0.5
Coefficients: [0.7367173713237881,0.023777396068750935,1.717518301535294,586865.9572014769]
R Squared (R2) on test data = 0.139842
month2
0.01
0.0
Coefficients: [0.9221224574835202,0.04935703984054278,2.8903468234122918,533196.4068670084]
R Squared (R2) on test data = 0.155964
month3
0.01
0.0
Coefficients: [1.0058898445410163,0.0536786119093709,3.1516967909530997,516966.1256172173]
R Squared (R2) on test data = 0.170174
month4
0.01
0.5
Coefficients: [1.096113869321891,0.04752320936891231,3.042040012551426,493391.452646429]
R Squared (R2) on test data = 0.181825
month5
0.01
0.0
Coefficients: [1.1494839595750486,0.0348699177238861,3.5568375328788235,489953.7516994188]
R Squared (R2) on test data = 0.19421
month6
0.01
0.0
Coefficients: [1.1932678820089453,0.028771790576247354,3.564650844999889,474759.2352543097]
R Squared (R2) on test data = 0.207958
month7
0.01
0.0
Coefficients: [1.0102397915763963,0.023463846484572056,4.459863048569217,564510.57179736]
R Squared (R2) on test data = 0.232848
month8
0.01
0.0
Coefficients: [1.1957827562061862,0.033209421569307285,3.7026093278739474,498315.3547475684]
R Squared (R2) on test data = 0.235154
month9
0.01
0.5
Coefficients: [1.2570213961573269,0.001271112015650849,3.406488158496962,458110.12487776723]
R Squared (R2) on test data = 0.225946
month10
0.01
0.5
Coefficients: [1.2594069763229117,0.026555413940664963,2.8838426813895337,428949.80224280985]
R Squared (R2) on test data = 0.219659
month11
0.01
0.0
Coefficients: [1.2112293557448608,0.1917545529377824,3.1171769946213352,407326.2977484972]
R Squared (R2) on test data = 0.223918
month12
0.01
0.0
Coefficients: [1.201934229551218,0.15411952043832208,2.496858595801839,377828.056797545]
R Squared (R2) on test data = 0.214864

In [24]:
import matplotlib.pyplot as plt

month_list = [0,1,2,3,4,5,6,7,8,9,10,11,12]
MSE_list_SN  = MSE_list
fig, ax = plt.subplots()
ax.plot(month_list, MSE_list_SN,color='tab:blue',label='RF')

ax.set(xlabel='month of lifetime', ylabel='MSE',
       title=' MSE for each lifetime point - Social Network Metrics')
ax.grid()

plt.show()

## 5.4 Regression with social network metrics and the spending behavior profile metrics

In [26]:
venmo_reg_sn_1 = spark.read.parquet('/FileStore/tables/venmo_reg_sn_1.parquet')
accum_profile_prop = spark.read.parquet('/FileStore/tables/accum_profile_prop.parquet')
venmo_reg_sn_sb = venmo_reg_sn_1.join(accum_profile_prop,['user','month'],how ='inner').orderBy('user','month').cache()

In [27]:
# Prepare for user and friend table with month interval
venmo_data = spark.read.parquet('/FileStore/tables/VenmoSample_snappy-e020d.parquet')
venmo_data.createOrReplaceTempView('venmo_data')

dynamic_friending = spark.sql("SELECT distinct user, friend, datetime\
                               FROM  \
                               (SELECT user1 as user, user2 as friend, datetime \
                               FROM venmo_data \
                               UNION \
                               SELECT user2 as user, user1 as friend, datetime \
                               FROM venmo_data) ")
dynamic_friending.createOrReplaceTempView('dynamic_friending')

In [28]:
# create user and friend table with month interval
window = Window.partitionBy('user').orderBy('datetime')
dynamic_friending = dynamic_friending.withColumn("month_passed", F.ceil((F.datediff(dynamic_friending.datetime, 
                                   first(dynamic_friending.datetime).over(window)))/30) )

In [29]:
# Calculate the users' spending behavior of his/her social network
dynamic_friending.createOrReplaceTempView("dynamic_friending")
venmo_reg_sn_sb.createOrReplaceTempView("venmo_reg_sn_sb")
venmo_reg_sn_sb = spark.sql("""
                SELECT v1.user, v1.month, v1.Y, v1.recency, v1.frequency, 
                v1.friend_degree, v1.fof_degree, v1.clustering_coefficient,
                v1.pagerank, 
                AVG(v2.activity_perc) as activity_perc,
                AVG(v2.food_perc) as food_perc,
                AVG(v2.illegal_perc) as illegal_perc,
                AVG(V2.others_perc) as others_perc,
                AVG(V2.people_perc) as people_perc,
                AVG(V2.utility_perc) as utility_perc,
                AVG(V2.cash_perc) as cash_perc,
                AVG(V2.event_perc) as event_perc,
                AVG(V2.transp_perc) as transp_perc,
                AVG(V2.travel_perc) as travel_perc
                FROM venmo_reg_sn_sb v1 LEFT JOIN dynamic_friending f
                ON v1.user = f.user 
                LEFT JOIN venmo_reg_sn_sb v2
                ON f.friend = v2.user
                group by v1.user, v1.month, v1.Y, v1.recency, v1.frequency, 
                v1.friend_degree, v1.fof_degree, v1.clustering_coefficient,
                v1.pagerank
                """)

In [30]:
# Change the dataframe into the model suitable format
vectorAssembler = VectorAssembler(inputCols = ['friend_degree','fof_degree','clustering_coefficient','pagerank','activity_perc','food_perc','illegal_perc','others_perc','people_perc','utility_perc','cash_perc','event_perc','transp_perc','travel_perc'], outputCol = 'features')
venmo_reg_sn_sb_agg = vectorAssembler.transform(venmo_reg_sn_sb)
venmo_reg_sn_sb_agg.write.parquet('venmo_reg_sn_sb_agg.parquet',mode='overwrite')

In [31]:
venmo_reg_sn_sb_agg = spark.read.parquet('venmo_reg_sn_sb_agg.parquet')
# Train and test split
for i in range(0,13):
    splits = venmo_reg_sn_sb_agg[venmo_reg_sn_sb_agg['month']==i].randomSplit([0.7, 0.3])
    train = splits[0].select(['features','Y'])
    test = splits[1].select(['features','Y'])
    train_df = train.withColumnRenamed('Y','label')
    test_df = test.withColumnRenamed('Y','label')
    train_df.write.mode('overwrite').format("parquet").save('dbfs:/FileStore/tables/train_sn_sb_month'+str(i)+'.parquet')
    test_df.write.mode('overwrite').format("parquet").save('dbfs:/FileStore/tables/test_sn_sb_month'+str(i)+'.parquet')

In [32]:
MSE_list = []
for i in range(0,13):  
    # For each user’s lifetime point, regress his/her social network metrics and the spending behavior of her social network on Y.
    lr = LinearRegression(featuresCol='features', labelCol='label', maxIter=10)

    # We use a ParamGridBuilder to construct a grid of parameters to search over.
    # TrainValidationSplit will try all combinations of values and determine best model using the evaluator.
    paramGrid = ParamGridBuilder()\
        .addGrid(lr.regParam, [0.1, 0.01]) \
        .addGrid(lr.fitIntercept, [False, True])\
        .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])\
        .build()

    # A TrainValidationSplit requires an Estimator, a set of Estimator ParamMaps, and an Evaluator.
    tvs = TrainValidationSplit(estimator=lr,
                               estimatorParamMaps=paramGrid,
                               evaluator=RegressionEvaluator(),
                               # 80% of the data will be used for training, 20% for validation.
                               trainRatio=0.8)

    # Run TrainValidationSplit, and choose the best set of parameters.
    train_df = spark.read.parquet("/FileStore/tables/train_sn_sb_month"+str(i)+".parquet")
    test_df = spark.read.parquet("/FileStore/tables/test_sn_sb_month"+str(i)+".parquet")
    model = tvs.fit(train_df)
    
    # Out of Sample Mean Squared Error
    lr_predictions = model.transform(test_df)
    evaluator = RegressionEvaluator(metricName="mse")
    MSE = evaluator.evaluate(lr_predictions)
    MSE_list.append(MSE)
    
    #
    print(i)
    print("Coefficients: " + str(model.bestModel.coefficients))

/databricks/spark/python/pyspark/ml/util.py:791: UserWarning: Can not find mlflow. To enable mlflow logging, install MLflow library from PyPi.
 warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)
0
Coefficients: [1.1770529524796969,-0.1607704085745203,-0.5924252491356166,1194033.3969929416,2.415842066660732,5.708771602247543,6.243646980278579,4.632706902713176,4.232383075787641,2.9714361661190205,3.8203667807112858,1.8658372984758154,5.367690151541615,0.8816428423269913]
1
Coefficients: [0.6611745151509072,0.014363324909350323,1.868820617938556,506750.07048183016,1.655960152947969,4.541505945247291,4.628499755403122,3.509845029700969,2.9104730273315798,2.505538414173456,2.676692605890246,1.2118773371602143,4.113214922683972,-0.1294628323260239]
2
Coefficients: [0.8334154913000922,0.03268706261783396,2.7010708784209636,456037.72743736824,1.6602913054186101,4.562437377641785,4.754583853331629,3.407809639478323,2.7620791364344743,1.986843002925091,3.0431645835254426,1.1004307912944369,4.037790378814338,-0.3199936146082582]
3
Coefficients: [0.9305264192905108,0.03558325825970875,2.8397674609905437,438687.2367916032,1.5354763185744127,4.392241050272315,4.504892708390646,3.312721918699386,2.6137208152899434,1.9190501755132352,2.6621072709990896,0.9144605793639775,4.008621387111709,-0.33614926285724667]
4
Coefficients: [1.006569416681292,0.03917949421888327,3.013072072773633,425180.1951735625,1.4224250662496876,4.392806734534136,4.467224175841667,3.2058745443972922,2.520784167143006,1.630553048731384,2.930268480635643,0.6678066417276619,3.8879230607880544,-0.7410677744282781]
5
Coefficients: [0.9975651024302437,0.032082607601504456,3.5187939798707646,410509.32091856824,1.2742377126935613,4.098494302653909,4.359151047315848,2.991407217395976,2.44232291305444,1.634166125763065,2.618772834010874,0.78647428387242,3.7751439759435037,-0.32716080768879746]
6
Coefficients: [1.0708814030389269,0.019328999585722036,3.500035342401226,413881.8662668815,1.121771624446044,4.007150656844377,4.1286442115021345,2.980550964389022,2.3043550554277124,1.546464733099983,2.559836633992535,0.6131207162976794,3.695665256307088,-0.43223064520260035]
7
Coefficients: [0.9459523878138275,0.017878234255606473,4.144772871978628,495240.12193426053,1.0247905641613435,3.8893884730507846,3.877635502207489,2.852818923350813,2.2523548795453388,1.4122594747306694,2.3942035415136127,0.3967405845679141,3.4590690188774826,-0.3906016713474364]
8
Coefficients: [1.0822097211136386,0.03878255092614151,3.3849509009050673,426673.9626566085,0.8578643437259393,3.6903751650189602,3.9883998579440174,2.743408418767772,2.1934463570141873,1.280547602395005,2.633643606801675,0.40732910832842734,3.4118001224056194,-0.5488352868315837]
9
Coefficients: [1.1410455984790355,0.0021904192633740347,3.207962335674121,391026.8639133959,0.8547658763417771,3.732790406652348,4.123548814866742,2.8399102599612354,2.3205694763095934,1.2011270141536223,2.6218395474823035,0.38667677795309247,3.534696621969471,-0.5060076982526309]
10
Coefficients: [1.1858002787647737,0.0206148718943501,2.454157850722663,371665.6263571756,0.8610719253169751,3.7708129138186686,4.152222233359191,2.9544692038395355,2.4267872346445287,1.2778146880763237,2.397580487948736,0.17975734842116137,3.443636103546493,-0.41824444593918697]
11
Coefficients: [1.1016628584858061,0.16232764959003693,2.7859620782045122,345531.0833722388,0.8475289945217576,3.60655878627629,4.084692186383762,2.8163628582206677,2.357848100244682,1.3735639073827604,2.4495208424282864,0.14989146171255358,3.407352191353195,-0.5116062215571879]
12
Coefficients: [1.1274675722136418,0.11634854002160758,2.397418451603155,325561.11656087195,0.7252235343163532,3.703459867598256,4.014424882947589,2.9598224361967245,2.499301391180538,1.6056083357454558,2.268894035697496,0.37225045614375674,3.4597359391768214,0.016850326576453467]

In [33]:
import matplotlib.pyplot as plt

month_list = [0,1,2,3,4,5,6,7,8,9,10,11,12]
MSE_list_SN_SB  = MSE_list
fig, ax = plt.subplots()
ax.plot(month_list, MSE_list_SN_SB,color='tab:blue',label='RF')

ax.set(xlabel='month of lifetime', ylabel='MSE',
       title=' MSE for each lifetime point - Social Network Metrics and Spending Profile')
ax.grid()

plt.show()